### 실습 개요
- 특정 유사도 수치를 기준으로 Video array를 분할하고 그룹화(Grouping)합니다.
- 그룹별 평균 유사도를 활용하여 해당 그룹의 활동성을 분석합니다.
- 분석 내용을 csv 파일에 저장합니다.

### 사전준비
- functions.py 안에 mp4 파일 decoding 함수들을 import 한다

In [ ]:
# functions.py 에 미리 정의된 함수들을 import 한다
import functions as fs

# numpy를 import 한다
import numpy as np

- mp4 file로부터 video array, 재생시간, frame개수 정보를 추출하기

In [ ]:
"""
함수 video_2_ndarray 를 사용하여 ../media/SampleVideo_640x360_5mb.mp4 의 video data를 ndarray 에 저장한다.
video array, 재생시간, frame 개수를 저장
"""
video_array, tot_duration, tot_frames = fs.video_2_ndarray('../media/SampleVideo_640x360_5mb.mp4')

- 연속되는 두 frame들간의 유사도를 구해서 list 변수(similarity_list)에 저장

In [ ]:
from numpy import dot
from numpy.linalg import norm

# cosine similatiry 함수
def cos_sim(A, B):
    return dot(A, B) / (norm(A) * norm(B))

In [ ]:
prev_vector = None

similarity_list = []

for frame in video_array:

    # 1차원으로 reshape
    # 255로 나눔
    current_vector = (frame.reshape(-1) / 255)
    
    if prev_vector is not None:
        similarity = cos_sim(prev_vector, current_vector)
        similarity_list.append(similarity)
            
    prev_vector = current_vector.copy()

print('similarity_list : ',similarity_list)

- pandas를 import 한다

In [ ]:
import pandas as pd

### 따라 해보기. pandas를 사용하여 영상 유사도 list를 grouping 하기
- 유사도가 낮아지는 지점을 기준으로 grouping 하기

#### 1. 유사도 DataFrame 생성

In [ ]:
# DataFrame 생성
df = pd.DataFrame(similarity_list, columns=['similarity'])

print(df)

In [ ]:
df.plot()

#### 2. 특정 유사도 값을 기준으로 '낮은 유사도'와 '높은 유사도' 두그룹으로 분할하기
-  ![Alt text for broken image link](../resources/pandas_plot.jpg)

In [ ]:
# 유사도 임계치 설정
lower_sim_threshold = 0.9

#### 3. 유사도 DataFrame에 2개 칼럼 추가
- lower_sim : 유사도 임계치 이하 여부

In [ ]:
# 유사도가 임계치 이하면 True(1) 아니면 False(0)
df['lower_sim'] = (df['similarity'] < lower_sim_threshold)

# 출력 시 '...' 없이 전체 출력
pd.set_option('display.max_rows', None)

# df 출력
print(df)

- cumsum : lower_sim의 누적 합계

In [ ]:
#lower_sim의 누적합(Cumulative Sum)을 구함
df['cumsum'] = df['lower_sim'].cumsum()

# df 출력
print(df)

# 출력 시 30 rows 출력 후 '...' 처리
pd.set_option('display.max_rows', 30)

In [ ]:
df.plot()

#### 4. cumsum값 을 기준으로 grouping
- 결과적으로 유사도가 낮은 순간(lower_sim_threshold 보다 작은 순간) group이 분할됨

In [ ]:
df_takes = df.groupby(df['cumsum'])

for group_number, group_df in df_takes:
    print(f"Group : {group_number}   frame Count : {group_df.shape[0]}")

#### 5. Group별 시작 frame 번호, 종료 frame 번호 조회

In [ ]:
for group_number, group_df in df_takes:
    # 시작 frame 번호
    min_frame_number = group_df.index.min()+1
    # 종료 frame 번호
    max_frame_number = group_df.index.max()+1

    print(f"Group : {group_number}, 시작 frame 번호 : {min_frame_number}, 종료 frame 번호 : {max_frame_number}")

#### 6. 캐릭터의 Group별 활동량 계산
- 유사도 평균은 해당 Group의 영상이 얼마나 움직임이 많은지를 가늠함
  - 이상치 data 제거(outlier removal)
  - 좁은 범위에 조밀하게 분포한 값을 넓게 분포하도록 거듭제곱 변환 (Power Transformation)사용
  - 1−x 변환을 이용해서 데이터의 범위를 [0, 1]로 유지하면서 값의 관점을 반대로 뒤집음

In [ ]:
mean = []
mean_power = []
activity = []

for group_number, group_df in df_takes:

    # 1. 이상치 Data 제거(outlier removal)
    # 영상의 Take가 변경되는 지점의 similarity를 제외(similarity > 0.9)
    refined_df =  group_df[group_df['similarity'] > 0.9]

    # 해당 Take(group DataFrame)의 similarity 평균을 구함
    similarity_mean = refined_df['similarity'].mean()

    # 2. 거듭제곱 변환 (Power Transformation)
    # 좁은 범위에 조밀하게 분포한 값을 넓게 분포하도록 변환(100 거듭제곱)
    # - data들이 0~1 사이 값이므로 거듭제곱 할수록 1에서 멀어지고 0 에 가까와짐. 
    # - data들이 1 근처에 조밀하게 분포하는 경우 효과적으로 분산시킴
    similarity_mean_power = similarity_mean**100

    # 3. 1−x 변환
    # 데이터의 범위를 [0, 1]로 유지하면서 값의 관점을 반대로 뒤집음
    # - 값이 클수록 동적인 영상, 작을수록 정적인 영상이됨
    activity_intensity = 1 - similarity_mean_power
    
    mean.append(similarity_mean)
    mean_power.append(similarity_mean_power)
    activity.append(activity_intensity)

In [ ]:
# (참고) 컴프리헨션 방식으로 계산
# 그룹별 유사도 평균 
mean = [ group_df[group_df['similarity'] > 0.9]['similarity'].mean() \
        for _, group_df in df_takes ]

# 거듭제곱 변환 (100 거듭제곱)
mean_power = [ v**100 for v in mean ]

# 1-x 변환
activity = [ 1-v for v in mean_power]

- 시각화하여 차이를 확인하기

In [ ]:
pd.DataFrame({'mean':mean, 'mean_power':mean_power, 'activity':activity}).plot()

#### 7. Group별 frame개수, 시작시간, 재생시간 계산
- 사전준비 단계에서 구했던 tot_duration(재생시간), tot_frames(frame 개수) 사용하여 frame개수, 시작시간, 재생시간 항목 생성

In [ ]:
frame_count = []
start = []
duration = []

for group_number, group_df in df_takes:

    min_frame_number = group_df.index.min()+1
    max_frame_number = group_df.index.max()+1

    v_frame_count = max_frame_number - min_frame_number + 1

    # 사전준비 단계에 구했던 tot_duration(재생시간), tot_frames(frame 개수) 사용
    v_start = tot_duration*(min_frame_number/tot_frames)
    v_duration = tot_duration*(v_frame_count/tot_frames)    
    
    print(f"group no :{group_number}, frame 개수 :{v_frame_count}, start :{v_start:.2f}, duration :{v_duration:.2f}")

    frame_count.append(v_frame_count)
    start.append(v_start)
    duration.append(v_duration)

#### 8. Group DataFrame 생성

In [ ]:
# group 정보로 DataFrame 생성
group_df = pd.DataFrame(
    {
        'frame_count': frame_count,
        'start': start,
        'duration': duration,
        'activity': activity
    }
)

# 인덱스 칼럼명 지정
group_df.index.name = "group_number"

print(group_df)

#### 9. DataFrame을 csv파일에 저장

In [ ]:
csv = group_df.to_csv(
        path_or_buf = "video_grouping.csv", 
        columns = ['frame_count', 'start', 'duration', 'activity'], 
        index = True,
    )

#### 10. Web 으로 공개하기

In [ ]:
def process_video(file_path):

    def cos_sim(A, B):
        return dot(A, B) / (norm(A) * norm(B))

    # 비디오 파일 -> ndarray 변환
    video_array, tot_duration, tot_frames = fs.video_2_ndarray(file_path)

    # 프레임 간 유사도 계산
    similarity_list = []
    for frame_1, frame_2 in zip(video_array[:-1], video_array[1:]):
        vector_1 = frame_1.reshape(-1) / 255.0
        vector_2 = frame_2.reshape(-1) / 255.0
        similarity = cos_sim(vector_1, vector_2)
        similarity_list.append(similarity)

    # 유사도 DataFrame 생성
    df = pd.DataFrame(similarity_list, columns=['similarity'])
    
    # 유사도가 임계치 이하면 True(1) 아니면 False(0)
    df['lower_sim'] = (df['similarity'] < lower_sim_threshold)
    
    # lower_sim의 누적합(Cumulative Sum)을 구함
    df['cumsum'] = df['lower_sim'].cumsum()
    
    # cumsum값을 기준으로 grouping
    df_takes = df.groupby(df['cumsum'])
    
    # Group별 frame개수, 시작시간, 재생시간, 활동량 계산
    frame_count = []
    start = []
    duration = []
    activity = []

    for group_number, group_df in df_takes:

        min_frame_number = group_df.index.min() + 1
        max_frame_number = group_df.index.max() + 1
        v_frame_count = max_frame_number - min_frame_number + 1
        v_start = tot_duration * (min_frame_number / tot_frames)
        v_duration = tot_duration * (v_frame_count / tot_frames)

        refined_df = group_df[group_df['similarity'] > lower_sim_threshold]
        similarity_mean = refined_df['similarity'].mean()
        similarity_mean_power = similarity_mean**100
        v_activity_intensity = 1 - similarity_mean_power
        
        frame_count.append(v_frame_count)
        start.append(v_start)
        duration.append(v_duration)
        activity.append(v_activity_intensity)

    # Group DataFrame 생성
    group_df = pd.DataFrame({
        'frame_count': frame_count,
        'start': start,
        'duration': duration,
        'activity': activity
    })
    
    return group_df.to_html(), file_path

In [ ]:
import gradio as gr

# Gradio 인터페이스
iface = gr.Interface(
    fn=process_video, 
    inputs=gr.File(label="동영상 파일을 업로드하세요"),
    outputs=[
        gr.HTML(label="유사도 분석 결과"),   # 첫 번째 출력: HTML
        gr.Video(label="동영상 재생")      # 두 번째 출력: Video
    ],
    title="비디오 분석 & 유사도 결과",
    description="업로드한 동영상의 연속 프레임 간 유사도(코사인 유사도)를 계산하여, 통계치 및 액티비티를 표로 보여줍니다."
)

# 실행
iface.launch()

### 생각 해보기
활동량(Activity)을 지표화하는 과정에서 이상치 제거(Outlier Removal), 거듭제곱 변환(Power Transformation), 그리고 (1−x) 변환을 적용하는 이유와 효과는 각각 무엇일까?
- 이상치 제거는 왜 필요하며, 제거하지 않았을 때 어떤 문제가 발생할 수 있을까?
- 거듭제곱 변환을 통해 얻을 수 있는 이점은 무엇일까? (예: 데이터 분포의 왜곡 완화 등)
- 최종적으로 (1−x) 변환을 적용하는 의도는 무엇일까?